In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns

import torch

from src.utils import haversine, radians
from dataset import ObservationsDataset

%load_ext autoreload
%autoreload 2

In [2]:
legend_df = pd.read_csv('data/subway_locations_connections.csv')

def parse_string_array_frame(df, column):
    df[column] = df[column].apply(lambda x : [str(i.strip().upper()[1:-1]) for i in x[1:-1].split(',')])

def parse_float_array_frame(df, column):
    df[column] = df[column].apply(lambda x : [float(i.strip()) for i in x[1:-1].split(',')])

parse_string_array_frame(legend_df, 'dest_name')
parse_string_array_frame(legend_df, 'dest_id')
parse_float_array_frame(legend_df, 'dest_lat')
parse_float_array_frame(legend_df, 'dest_long')

origin_cols = [x for x in legend_df.columns if x.startswith('origin')]
dest_cols = [x for x in legend_df.columns if x.startswith('dest')]

# legend_df[['origin_lat', 'origin_long', 'dest_lat', 'dest_long']] = \
#     legend_df[['origin_lat', 'origin_long', 'dest_lat', 'dest_long']].applymap(lambda x : radians(x))

nodes_df = legend_df[origin_cols]

In [3]:
nodes_df = nodes_df.rename(columns=lambda x : x[len('origin_'):] if x.startswith('origin_') else x)
nodes_df = nodes_df.rename(columns={'long':'lon'})
nodes_df['id'] = nodes_df['name']
nodes_df = nodes_df.set_index('name')
nodes_df

,id,lat,lon
name,,,
1 AV,1 AV,40.730953,-73.981628
103 ST,103 ST,40.799446,-73.968379
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,40.749865,-73.862700
104 ST,104 ST,40.681711,-73.837683
110 ST,110 ST,40.795020,-73.944250
...,...,...,...
WOODLAWN,WOODLAWN,40.886037,-73.878751
WOODSIDE - 61 ST,WOODSIDE - 61 ST,40.745630,-73.902984
WORLD TRADE CENTER,WORLD TRADE CENTER,40.712582,-74.009781


In [4]:
import geopandas as gpd 
from shapely import wkt
from shapely.geometry import Point

nodes_gdf = gpd.GeoDataFrame(nodes_df, crs='epsg:4326')
nodes_gdf = nodes_gdf.set_geometry(
    [Point(row['lon'], row['lat']) for i, row in nodes_gdf[['lat', 'lon']].iterrows()]
)

voronoi_data = pd.read_csv('data/subway_data_voronoi.csv')
voronoi_data['geometry'] = voronoi_data['geometry'].apply(wkt.loads)
voronoi_gdf = gpd.GeoDataFrame(voronoi_data, geometry='geometry', crs='epsg:4326')
nodes_gdf = gpd.sjoin_nearest(nodes_gdf, voronoi_gdf[['geometry', 'population', 'jobs']])
nodes_df = pd.DataFrame(nodes_gdf.drop(columns=['geometry', 'index_right']))
nodes_df = nodes_df[nodes_df['population'] != 0]

/home/eldervald/miniconda3/envs/traffic/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
/home/eldervald/miniconda3/envs/traffic/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  """
/home/eldervald/miniconda3/envs/traffic/lib/python3.7/site-packages/geopandas/array.py:348: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  stacklevel=stacklevel,


In [5]:
emp = pd.read_csv('data/mta_employment.csv').set_index('origin_name')
emp = emp[emp.columns[-1]].rename('labor_force_rate')
emp = emp.replace('-', None).astype(float) / 100.0
nodes_df = nodes_df.join(emp)
nodes_df

,id,lat,lon,population,jobs,labor_force_rate
name,,,,,,
1 AV,1 AV,40.730953,-73.981628,63146,5499.0,0.734
103 ST,103 ST,40.799446,-73.968379,24164,1100.0,0.641
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,40.749865,-73.862700,48310,706.0,0.718
104 ST,104 ST,40.681711,-73.837683,9781,1656.0,0.622
110 ST,110 ST,40.795020,-73.944250,21003,3590.0,0.619
...,...,...,...,...,...,...
WOODLAWN,WOODLAWN,40.886037,-73.878751,9571,67.0,0.609
WOODSIDE - 61 ST,WOODSIDE - 61 ST,40.745630,-73.902984,27095,7632.0,0.672
WORLD TRADE CENTER,WORLD TRADE CENTER,40.712582,-74.009781,806,0.0,0.783


In [6]:
house_pricing = pd.read_csv('data/mta_housing_salePrice.csv').set_index('origin_name')
house_pricing.drop(columns=[x for x in house_pricing.columns if not x.startswith('Estimate')], inplace=True)
house_pricing.rename(columns= lambda x: 'housing_sales_num_' + x[len('estimate!!'):], inplace=True)
house_pricing = house_pricing.astype(float) / 100.0
house_pricing.fillna(0, inplace=True)
nodes_df = nodes_df.join(house_pricing)
nodes_df

,id,lat,lon,population,jobs,labor_force_rate,housing_sales_num_Total:,"housing_sales_num_Total:!!Less than $10,000","housing_sales_num_Total:!!$10,000 to $14,999","housing_sales_num_Total:!!$15,000 to $19,999",...,"housing_sales_num_Total:!!$175,000 to $199,999","housing_sales_num_Total:!!$200,000 to $249,999","housing_sales_num_Total:!!$250,000 to $299,999","housing_sales_num_Total:!!$300,000 to $399,999","housing_sales_num_Total:!!$400,000 to $499,999","housing_sales_num_Total:!!$500,000 to $749,999","housing_sales_num_Total:!!$750,000 to $999,999","housing_sales_num_Total:!!$1,000,000 to $1,499,999","housing_sales_num_Total:!!$1,500,000 to $1,999,999","housing_sales_num_Total:!!$2,000,000 or more"
name,,,,,,,,,,,,,,,,,,,,,
1 AV,1 AV,40.730953,-73.981628,63146,5499.0,0.734,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00
103 ST,103 ST,40.799446,-73.968379,24164,1100.0,0.641,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,40.749865,-73.862700,48310,706.0,0.718,0.33,0.0,0.0,0.0,...,0.0,0.0,0.0,0.09,0.0,0.09,0.15,0.0,0.00,0.00
104 ST,104 ST,40.681711,-73.837683,9781,1656.0,0.622,0.26,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.13,0.00
110 ST,110 ST,40.795020,-73.944250,21003,3590.0,0.619,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOODLAWN,WOODLAWN,40.886037,-73.878751,9571,67.0,0.609,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00
WOODSIDE - 61 ST,WOODSIDE - 61 ST,40.745630,-73.902984,27095,7632.0,0.672,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00
WORLD TRADE CENTER,WORLD TRADE CENTER,40.712582,-74.009781,806,0.0,0.783,0.57,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.57


In [7]:
emp = pd.read_csv('data/mta_median_income.csv').set_index('origin_name')
median_income = emp['Estimate!!Median income (dollars)!!FAMILIES!!Families'].rename('median_income')
median_income = median_income[median_income != 0]
nodes_df = nodes_df.join(median_income, how='inner')
nodes_df.head(5)

,id,lat,lon,population,jobs,labor_force_rate,housing_sales_num_Total:,"housing_sales_num_Total:!!Less than $10,000","housing_sales_num_Total:!!$10,000 to $14,999","housing_sales_num_Total:!!$15,000 to $19,999",...,"housing_sales_num_Total:!!$200,000 to $249,999","housing_sales_num_Total:!!$250,000 to $299,999","housing_sales_num_Total:!!$300,000 to $399,999","housing_sales_num_Total:!!$400,000 to $499,999","housing_sales_num_Total:!!$500,000 to $749,999","housing_sales_num_Total:!!$750,000 to $999,999","housing_sales_num_Total:!!$1,000,000 to $1,499,999","housing_sales_num_Total:!!$1,500,000 to $1,999,999","housing_sales_num_Total:!!$2,000,000 or more",median_income
1 AV,1 AV,40.730953,-73.981628,63146,5499.0,0.734,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,83750
103 ST,103 ST,40.799446,-73.968379,24164,1100.0,0.641,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,195250
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,40.749865,-73.862700,48310,706.0,0.718,0.33,0.0,0.0,0.0,...,0.0,0.0,0.09,0.0,0.09,0.15,0.0,0.00,0.0,42328
104 ST,104 ST,40.681711,-73.837683,9781,1656.0,0.622,0.26,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.13,0.0,69375
110 ST,110 ST,40.795020,-73.944250,21003,3590.0,0.619,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,33323


In [8]:
# population = pd.read_csv('data/mta_population.csv').set_index('origin_name')
# population.drop(columns=[x for x in population.columns if not x.startswith('Estimate')], inplace=True)
# population.rename(columns= lambda x: x[len('estimate!!population!! '):], inplace=True)
# population = population[population != 0]
# population.fillna(0, inplace=True)
# nodes_df = nodes_df.join(population, how='inner')
# nodes_df.head(5)

In [9]:
def make_graph_from_df(nodes_df, edges, name='TLC', directed=True):
    G = nx.Graph(directed=directed)
    G.graph['Name'] = name

    G.add_nodes_from(nodes_df.to_dict('index').items())
    G.add_edges_from(edges)
    G.remove_edges_from(nx.selfloop_edges(G))

    return G

In [10]:
mta = pd.read_csv('data/subway_2021_ridership.csv')
mta = mta[mta['flag'] == True]
mta['DATE_time'] = pd.to_datetime(mta['DATE_time'])
mta['DATE'] = mta['DATE_time']
mta = mta.groupby(['DATE', 'STATION'], as_index=False)['ridership'].sum()
mta['ridership'] = mta['ridership'].astype('float32')
mta

,DATE,STATION,ridership
0,2020-12-26 00:00:00,82 ST-JACKSON H,6198.0
1,2020-12-26 00:00:00,YORK ST,6928.0
2,2020-12-26 04:00:00,1 AV,3.0
3,2020-12-26 04:00:00,103 ST,1.0
4,2020-12-26 04:00:00,103 ST-CORONA,81.0
...,...,...,...
610870,2021-12-24 20:00:00,WOODHAVEN BLVD,396.0
610871,2021-12-24 20:00:00,WOODLAWN,55.0
610872,2021-12-24 20:00:00,WORLD TRADE CTR,41.0
610873,2021-12-24 20:00:00,YORK ST,368.0


In [11]:
mean_target = mta.groupby('STATION').mean().rename(columns={'ridership': 'mean_target'})
mean_weekend_target = mta[mta['DATE'].dt.weekday.isin((5, 6))].groupby(['STATION']).mean() \
    .rename(columns={'ridership': 'mean_weekend_target'})
mean_workday_target = mta[~mta['DATE'].dt.weekday.isin((5, 6))].groupby(['STATION']).mean() \
    .rename(columns={'ridership': 'mean_workday_target'})

nodes_df = nodes_df.join(mean_target, how='inner')
nodes_df = nodes_df.join(mean_weekend_target, how='inner')
nodes_df = nodes_df.join(mean_workday_target, how='inner')
nodes_df

,id,lat,lon,population,jobs,labor_force_rate,housing_sales_num_Total:,"housing_sales_num_Total:!!Less than $10,000","housing_sales_num_Total:!!$10,000 to $14,999","housing_sales_num_Total:!!$15,000 to $19,999",...,"housing_sales_num_Total:!!$400,000 to $499,999","housing_sales_num_Total:!!$500,000 to $749,999","housing_sales_num_Total:!!$750,000 to $999,999","housing_sales_num_Total:!!$1,000,000 to $1,499,999","housing_sales_num_Total:!!$1,500,000 to $1,999,999","housing_sales_num_Total:!!$2,000,000 or more",median_income,mean_target,mean_weekend_target,mean_workday_target
1 AV,1 AV,40.730953,-73.981628,63146,5499.0,0.734,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,83750,83.161156,58.006451,93.171371
103 ST,103 ST,40.799446,-73.968379,24164,1100.0,0.641,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,195250,132.695175,94.853462,147.817245
104 ST,104 ST,40.681711,-73.837683,9781,1656.0,0.622,0.26,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.13,0.00,69375,58.162685,29.911291,69.419662
110 ST,110 ST,40.795020,-73.944250,21003,3590.0,0.619,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,33323,74.474266,42.242718,87.259308
111 ST,111 ST,40.751730,-73.855334,33789,1743.0,0.713,0.08,0.0,0.0,0.0,...,0.0,0.0,0.00,0.08,0.00,0.00,50759,51.094402,28.783033,59.929588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WINTHROP ST,WINTHROP ST,40.656652,-73.950200,17430,7395.0,0.638,0.17,0.0,0.0,0.0,...,0.0,0.0,0.00,0.17,0.00,0.00,75250,99.756096,49.950161,119.729851
WOODHAVEN BLVD,WOODHAVEN BLVD,40.733106,-73.869229,32419,6740.0,0.599,0.22,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,60000,226.677338,145.294785,259.124664
WOODLAWN,WOODLAWN,40.886037,-73.878751,9571,67.0,0.609,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,36275,123.176392,76.504837,141.761078
YORK ST,YORK ST,40.701397,-73.986751,10899,13956.0,0.880,0.84,0.0,0.0,0.0,...,0.0,0.0,0.45,0.00,0.00,0.39,250000,195.176285,145.030441,215.234619


In [12]:
nodes_df.dropna(inplace=True)
nodes_set = set(nodes_df.index.unique())
print(len(nodes_set))

mta = mta[mta['STATION'].isin(nodes_set)]

233


In [13]:
edges = []
for i, row in legend_df.iterrows():
    if row['origin_name'] not in nodes_set:
        continue
    
    for name, id, lat, lon in zip(*(row[col] for col in dest_cols)):
        # if name not in nodes_set:
        #     nodes_df = nodes_df.append(pd.DataFrame([[name, id, lat, lon]], columns=origin_cols), ignore_index=True)
        #     nodes_set.add(name)
        if name not in nodes_set:
            continue

        olon, olat = row['origin_long'], row['origin_lat']
        dist = float(haversine(olon, olat, lon, lat))

        if dist < 1e-8:
            continue

        edges.append((row['origin_name'], name, {'dist' : dist}))

edges[:5]
        

[('1 AV', '3 AV', {'dist': 30.919180852451866}),
 ('1 AV', 'BEDFORD AV', {'dist': 179.16600798420404}),
 ('103 ST', '96 ST', {'dist': 43.244356226697626}),
 ('103 ST', 'CATHEDRAL PKWY', {'dist': 30.409693548062044}),
 ('104 ST', '111 ST', {'dist': 38.53450231580343})]

In [14]:
for date in mta['DATE'].unique()[:30]:
    ds = ObservationsDataset.from_dataframe_by_day(mta, date)
    torch.save(ds, f'datasets/{date}.dat')

In [15]:
G = make_graph_from_df(nodes_df, edges)
print(G.number_of_nodes(), G.number_of_edges())
nx.readwrite.write_gpickle(G, 'data/network.gpickle')

233 221
